In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from comet_ml import Experiment
experiment = Experiment(api_key="S1cmDm3NaQEfL5WnfD4Z0IEdR",
                        project_name="general", workspace="cnarte")

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/cnarte/general/2eada09079a14118be7c4c89b9c3c6ba



In [2]:
import sklearn
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

In [3]:
traincsv = r"C:\Users\hp\Desktop\OSS COMP'\train.csv"
trf = pd.read_csv(traincsv)
trf.head()

,ID,Person A,Person B,Years of Knowing,Interaction Type,Interaction Duration,Moon Phase During Interaction,Friends
0,1,Paul,Deborah,4.72,Social_Media,0.00,Full Moon,1
1,2,Brian,Emily,9.67,In Passing,6.07,First Quarter,1
2,3,Kyle,Madison,1.64,In Passing,5.38,Waxing Gibbous,1
3,4,Daniel,Teresa,0.00,Over a Meal,15.72,Waxing Gibbous,0
4,5,Jordan,Megan,0.92,Over a Meal,12.36,Third Quarter,0


In [4]:
trf=trf.drop(['Person B','Person A','Moon Phase During Interaction'],1)
trf.head()

,ID,Years of Knowing,Interaction Type,Interaction Duration,Friends
0,1,4.72,Social_Media,0.00,1
1,2,9.67,In Passing,6.07,1
2,3,1.64,In Passing,5.38,1
3,4,0.00,Over a Meal,15.72,0
4,5,0.92,Over a Meal,12.36,0


In [5]:
ratingList = trf['Interaction Type'].tolist()

In [6]:
labelEncoderRating = LabelEncoder()
labelEncoderRating.fit(ratingList)
labelsRating = labelEncoderRating.transform(ratingList)
trf['Interaction Type']=pd.Series(labelsRating)

In [7]:
trf['Interaction Type'].describe()

count    54697.000000
mean         2.502057
std          1.705136
min          0.000000
25%          1.000000
50%          3.000000
75%          4.000000
max          5.000000
Name: Interaction Type, dtype: float64

In [8]:


    headers = list(trf)

#Store the revenues of movies separately in a list
    YIndex = headers.index('Friends')
    datasetMatrix = trf.as_matrix()
#Make the train and test splits
    datasetTrain = datasetMatrix[0:1500]
    datasetTrainWithoutLabels = np.delete(datasetTrain,YIndex,1)

    labels = datasetTrain[:,YIndex]

    datasetTest = datasetMatrix[1500:datasetMatrix.shape[0]]
    datasetTestWithoutLabels = np.delete(datasetTest,YIndex,1)

    trueLabels = datasetTest[:,YIndex]
    
    

E:\ANKIT\CONDA\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [9]:
regressor = Lasso()
alphas = np.arange(1,50)

In [10]:
steps = [('regressor',regressor)]
pipeline = Pipeline(steps)

In [11]:
parameterGrid = dict(regressor__alpha = alphas)
GridSearchResult = sklearn.model_selection.GridSearchCV(pipeline,param_grid=parameterGrid)

In [12]:
GridSearchResult.fit(datasetTrainWithoutLabels,labels)

E:\ANKIT\CONDA\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('regressor',
                                        Lasso(alpha=1.0, copy_X=True,
                                              fit_intercept=True, max_iter=1000,
                                              normalize=False, positive=False,
                                              precompute=False,
                                              random_state=None,
                                              selection='cyclic', tol=0.0001,
                                              warm_start=False))],
                                verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'regressor__alpha': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 4

In [13]:
print(GridSearchResult.best_params_)

{'regressor__alpha': 4}


In [14]:
predictions = GridSearchResult.predict(datasetTestWithoutLabels)
print (r2_score(trueLabels,predictions))

-0.00044001782586544635


In [15]:
print (predictions)

[0.496 0.496 0.496 ... 0.496 0.496 0.496]


In [16]:
from sklearn import metrics
print(metrics.accuracy_score(trueLabels,predictions))

ValueError: Classification metrics can't handle a mix of binary and continuous targets